# Data Hoarders

## Install

In [ ]:
#@title { display-mode: "form", form-width: "30%" }

rclone = True #@param {type: "boolean" }
speedtest = False #@param {type: "boolean" }
ssh = True #@param {type: "boolean"}

# Install libraries
if rclone:
  !curl https://rclone.org/install.sh &> /dev/null | sudo bash
  !rclone --version

if speedtest:
  !pip install speedtest-cli 2>&1
  !speedtest

if ssh:
  !ssh-keygen -q -t rsa -N '' -f ~/.ssh/id_rsa <<<y 2>&1 >/dev/null
  !curl -L https://github.com/tmate-io/tmate/releases/download/2.4.0/tmate-2.4.0-static-linux-i386.tar.xz &> /dev/null | tar xJC /usr/local/bin --strip-components 1 &> /dev/null
  !tmate -S /tmp/tmate.sock new-session -d &> /dev/null
  !sleep 1
  !tmate -S /tmp/tmate.sock display -p '#{tmate_ssh}'

## Rclone

### App Config

Set up `client_id` and `client_secret`

In [ ]:
#@title { display-mode: "form", run: "auto" }

client_id = "" #@param { type: "string" }
client_secret = "" #@param { type: "string" }

### Remotes

Create, add, update remote configs to be used elsewhere.

#### Drives

Remotes of Google Drive type.

In [ ]:
#@title { display-mode: "form" }

name = "source" #@param { type: "string" }
root_folder_id = "" #@param { type: "string" }
team_drive = "" #@param { type: "string" }
token = "" #@param { type: "string" }
service_account_file = "" #@param { type: "string" }
service_account_credentials = "" #@param { type: "string" }
chunk_size = "256M" #@param { type: "string" }

!rclone config delete {name}
!rclone config create {name} drive \
  client_id '{client_id}' \
  client_secret '{client_secret}' \
  config_is_local false \
  config_refresh_token false \
  token '{token}' \
  service_account_file '{service_account_file}' \
  service_account_credentials '{service_account_credentials}' \
  root_folder_id '{root_folder_id}' \
  team_drive '{team_drive}' \
  chunk_size {chunk_size} \
  server_side_across_configs true \
  stop_on_upload_limit true

In [ ]:
#@title { display-mode: "form" }

name = "target" #@param { type: "string" }
root_folder_id = "" #@param { type: "string" }
team_drive = "" #@param { type: "string" }
token = "" #@param { type: "string" }
service_account_file = "" #@param { type: "string" }
service_account_credentials = "" #@param { type: "string" }
chunk_size = "256M" #@param { type: "string" }

!rclone config delete {name}
!rclone config create {name} drive \
  client_id '{client_id}' \
  client_secret '{client_secret}' \
  config_is_local false \
  config_refresh_token false \
  token '{token}' \
  service_account_file '{service_account_file}' \
  service_account_credentials '{service_account_credentials}' \
  root_folder_id '{root_folder_id}' \
  team_drive '{team_drive}' \
  chunk_size {chunk_size} \
  server_side_across_configs true \
  stop_on_upload_limit true

#### Crypts

These crypt remotes encrypt and decrypt another remote.

In [ ]:
#@title { display-mode: "form" }

name = "crypt" #@param { type: "string" }
remote = "target:" #@param { type: "string" }
filename_encryption = "standard" #@param ["standard", "obfuscate", "off"]
directory_name_encryption = True #@param { type: "boolean" }
password = "" #@param { type: "string" }
password2 = "" #@param { type: "string" }
server_side_across_configs = True  #@param { type: "boolean" }
show_mapping = False #@param { type: "boolean" }

!rclone config delete {name}
!rclone config create {name} crypt \
  remote '{remote}' \
  filename_encryption '{filename_encryption}' \
  directory_name_encryption {directory_name_encryption} \
  password '{password}' \
  password2 '{password2}' \
  server_side_across_configs {server_side_across_configs} \
  show_mapping {show_mapping}

### Actions

#### Single remote

In [ ]:
#@title Drive { display-mode: "form" }

command = "lsd" #@param ["size", "lsd", "mkdir", "rmdir", "mount", "ncdu", "cat", "serve"]
name = "source:" #@param { type: "string" }
destination = "" #@param { type: "string" }
flags = "" #@param { type: "string" }

!rclone {command} {name} {destination} {flags}

#### Remote to Remote

In [ ]:
#@title Drive { display-mode: "form" }

command = "sync" #@param ["check", "copy", "sync", "move"]
source = "source:" #@param { type: "string" }
target = "crypt:" #@param { type: "string" }
buffer_size = "128M" #@param { type: "string" }
transfers = 15 #@param {type:"slider", min:5, max:30, step:5}
flags = "--progress --stats=5s --stats-one-line" #@param { type: "string" }

!rclone {command} '{source}' '{target}' \
  --transfers={transfers} \
  --cutoff-mode=soft \
  --buffer-size={buffer_size} \
  {flags}